In [1]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

/Users/lingyizhao/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
df_disambiguation = pd.read_parquet("hf://datasets/SALT-NLP/silent_signals_disambiguation/data/train-00000-of-00001.parquet")

/Users/lingyizhao/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df_disambiguation

,type,dog_whistle,dog_whistle_root,ingroup,content,date,speaker,chamber,reference,subreddit,label,definition
0,colloquial,terrorist,Islamic terrorists,Islamophobic,Radical feminism is a terrorist organization,None,None,None,None,walkaway,non-coded,Muslim people as a whole are a threat
1,colloquial,terrorist,Islamic terrorists,Islamophobic,There's a group of right-wing terrorists that ...,None,None,None,None,walkaway,non-coded,Muslim people as a whole are a threat
2,colloquial,terrorist,Islamic terrorists,Islamophobic,Would be difficult to do. About the only mecha...,None,None,None,None,consoom,non-coded,Muslim people as a whole are a threat
3,colloquial,terrorist,Islamic terrorists,Islamophobic,"Rapes, terrorist attacks, violence in general ...",None,None,None,None,walkaway,non-coded,Muslim people as a whole are a threat
4,colloquial,based,based,white supremacist,Where are the prosecutions for a doctor taking...,None,None,None,None,walkaway,non-coded,Endorsement of someone promoting racism or oth...
...,...,...,...,...,...,...,...,...,...,...,...,...
119,formal,election integrity,election integrity,racist,S. Senate spoke of ``the pernicious and nasty...,2020-01-06,Mr. BARRASSO,Senate,CREC-2021-03-03-pt1-PgS1007,None,noncoded,Voter suppression that targets people of color
120,formal,election integrity,election integrity,racist,"Earlier this year, we witnessed only the lates...",2020-01-06,Mr. SCHUMER,Senate,CREC-2021-03-17-pt1-PgS1578-2,None,noncoded,Voter suppression that targets people of color
121,formal,election integrity,election integrity,racist,One is the breakdown of trust in the system it...,2020-01-06,Mr. KING,Senate,CREC-2021-10-19-pt1-PgS7082-5,None,noncoded,Voter suppression that targets people of color
122,formal,election integrity,election integrity,racist,It was so disturbing to read in the Washington...,2020-01-06,Mr. SCHUMER,Senate,CREC-2023-02-01-pt1-PgS180-2,None,noncoded,Voter suppression that targets people of color


In [5]:
df_disambiguation.to_csv("df_disambiguation.csv", index=False)

In [12]:
from together import Together
import os
import json

In [13]:
os.environ["TOGETHER_API_KEY"] = "f32fc2d24a5ec68f0b440ad698afe50cf2d5baf6f8a3384d5e45c55abfff751e"

In [14]:
client = Together()

### 1 inference case

In [11]:
df_disambiguation_reorder = df_disambiguation.sample(frac=1, random_state=42).reset_index(drop=True)
df_disambiguation_grouped = df_disambiguation_reorder.groupby("dog_whistle")

In [15]:
results = []

for dog_whistle, group in df_disambiguation_grouped:
    sentences = group["sentence"].tolist()

    prompt = (
        f"You are a helpful assistant participating in a content moderation task of dog whistle detection.\n\n"
        f"A dog whistle is a coded message that conveys controversial or harmful ideas (often racist, sexist, or ideological) "
        f"in language that appears neutral to the general public but carries specific meaning to a targeted subgroup.\n\n"
        f'The coded meaning of "{dog_whistle}" is: {group["definition"]}.\n\n'
        f"Please examine the following sentences. Each sentence contains the keyword \"{dog_whistle}\". "
        f"Your task is to determine whether the keyword is being used as a dog whistle in each case.\n\n"
    )

    for i, sentence in enumerate(sentences, 1):
        prompt += f'{i}. "{sentence}"\n'

    prompt += (
    "\nPlease structure your response as a JSON object in the following format:\n"
    "{\n"
    "    {\n"
    "      \"sentence\": \" The full sentence here.\",\n"
    "      \"contains_dog_whistle\": true or false,\n"
    "      \"explanation\": \"If true, briefly explain how the dog whistle is used. Leave empty if false.\"\n"
    "    },\n"
    "    ... (repeat for each sentence)\n"
    "}"
    )

    # Send to LLM
    response = client.chat.completions.create(
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    result = response.choices[0].message.content

    results.append({
        "dog_whistle": dog_whistle,
        "prompt": prompt,
        "response": result
    })


KeyError: 'sentence'